In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures  #這用於生成多項式
#from cvxopt import matrix, solvers
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn_extra.cluster import KMedoids
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss, brier_score_loss
from scipy.stats import entropy
from sklearn.feature_selection import mutual_info_classif
import xgboost
import time
from tqdm import tqdm
import matplotlib.pylab as pylab

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

#solvers.options['show_progress'] = False

In [2]:
def print_hack(pd_data):
    print("|",end=" ")
    for i in pd_data.columns:
        print("|", i,end = " ")
    print("|")
    for i in range(pd_data.shape[1]+1):
        print("|---", end=" ")
    print("|")
    for i in range(pd_data.shape[0]):
        print("|",pd_data.index[i], end=" ")
        for j in pd_data.iloc[i,:]:
            print("|", j, end=" ")
        print("|")
        
def print_latex(pd_data):
    print('\\toprule')
    for i in pd_data.columns:
        print("&", i,end = " ")
    print('\\\\')
    print('\midrule')
    for i in range(pd_data.shape[0]):
        print(pd_data.index[i], end=" ")
        for j in pd_data.iloc[i,:]:
            print("&", j, end=" ")
        print('\\\\')
    print('\\bottomrule')


# Preprocessing

In [3]:
data = pd.read_csv('creditreform.csv',index_col=0)

data = data.reset_index().iloc[:,1:]

In [4]:
data_isna = pd.DataFrame(np.where(np.isnan(data[data.columns[data.isnull().sum()>0]]), 1, 0))

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(data.kurtosis())

VAR1     25220.481268
VAR2     25851.097490
VAR3     21648.591273
VAR4     33560.517797
VAR5      2019.593934
VAR6     17997.038634
VAR7     23532.717498
VAR8     16514.767755
VAR9     16679.961610
VAR10     3434.665159
VAR11    24164.631214
VAR12     8392.623169
VAR13     5109.099102
VAR14    29170.583471
VAR15    27215.026701
VAR16    23004.687609
VAR17    12759.540474
VAR18    18774.139304
VAR19    18780.128711
VAR20    10586.212651
VAR21    30554.317896
VAR22    36283.952475
VAR23    44148.193487
VAR24     9683.429944
T2          32.136280
dtype: float64


## Log transformation on three featrues

In [6]:
for i in [i for i in range(0,24)]:
    data.iloc[:,i] = np.log(data.iloc[:,i] + 1/(10**(-int(format(np.nanmean(data.iloc[:,i]),'.1E')[-3:])+6)))

Y = data.iloc[:,-1]
X = data.iloc[:,0:-1]

c:\Users\king0\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Standarization and impute mean

In [7]:
X = (X - X.mean()) / X.std()

X = X.fillna(0)

In [8]:
data = pd.concat([Y, X],axis = 1)

In [9]:
for i in range(2, 49, 2) :
    data.insert(i, str(i),data_isna.iloc[:,[(i/2)-1]])

In [10]:
data.columns = ["$y$","$x_{1}$","$x_{2}$","$x_{3}$","$x_{4}$","$x_{5}$","$x_{6}$","$x_{7}$","$x_{8}$","$x_{9}$","$x_{10}$", 
                      "$x_{11}$", "$x_{12}$", "$x_{13}$", "$x_{14}$", "$x_{15}$", "$x_{16}$", "$x_{17}$", "$x_{18}$", "$x_{19}$", "$x_{20}$", 
                      "$x_{21}$", "$x_{22}$", "$x_{23}$", "$x_{24}$", "$x_{25}$", "$x_{26}$", "$x_{27}$", "$x_{28}$", "$x_{29}$", "$x_{30}$", 
                      "$x_{31}$", "$x_{32}$", "$x_{33}$", "$x_{34}$", "$x_{35}$", "$x_{36}$", "$x_{37}$", "$x_{38}$", "$x_{39}$", "$x_{40}$", 
                      "$x_{41}$", "$x_{42}$", "$x_{43}$", "$x_{44}$", "$x_{45}$", "$x_{46}$", "$x_{47}$", "$x_{48}$"]

In [11]:
col = ['mean', 'std', 'min', '50%', 'max']
summary = data.describe()
na = pd.DataFrame(data.isnull().sum()/len(data)*100,columns = ['NA(%)']).T
skew = pd.DataFrame(data.skew(),columns=["skewness"]).T
kurt = pd.DataFrame(data.kurtosis(),columns=["kurtosis"]).T

In [12]:
print_latex(pd.concat([summary.loc[col], skew, kurt]).round(1).T)

\toprule
& mean & std & min & 50% & max & skewness & kurtosis \\
\midrule
$y$ & 0.0 & 0.2 & 0.0 & 0.0 & 1.0 & 5.8 & 32.1 \\
$x_{1}$ & -0.0 & 0.9 & -4.1 & 0.1 & 3.2 & -2.6 & 9.2 \\
$x_{2}$ & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 43.5 & 1892.9 \\
$x_{3}$ & 0.0 & 0.9 & -3.6 & 0.0 & 3.4 & -2.0 & 6.2 \\
$x_{4}$ & 0.2 & 0.4 & 0.0 & 0.0 & 1.0 & 1.2 & -0.5 \\
$x_{5}$ & -0.0 & 1.0 & -5.4 & 0.2 & 4.2 & -1.4 & 4.2 \\
$x_{6}$ & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 22.2 & 491.6 \\
$x_{7}$ & 0.0 & 0.9 & -3.9 & 0.0 & 3.6 & -1.4 & 4.0 \\
$x_{8}$ & 0.1 & 0.3 & 0.0 & 0.0 & 1.0 & 2.6 & 4.8 \\
$x_{9}$ & 0.0 & 0.9 & -2.3 & 0.0 & 2.8 & -1.2 & 2.1 \\
$x_{10}$ & 0.2 & 0.4 & 0.0 & 0.0 & 1.0 & 1.2 & -0.6 \\
$x_{11}$ & -0.0 & 1.0 & -5.9 & 0.1 & 4.3 & -1.2 & 3.6 \\
$x_{12}$ & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 38.9 & 1513.3 \\
$x_{13}$ & 0.0 & 1.0 & -4.4 & 0.1 & 3.9 & -1.7 & 5.2 \\
$x_{14}$ & 0.0 & 0.2 & 0.0 & 0.0 & 1.0 & 6.0 & 34.3 \\
$x_{15}$ & 0.0 & 0.8 & -3.4 & 0.0 & 3.1 & -1.7 & 6.6 \\
$x_{16}$ & 0.4 & 0.5 & 0.0 & 0.0 & 1.0 & 0

In [13]:
Xa = pd.concat([X, data_isna],axis = 1)

In [14]:
X_train_data, X_test_data, Y_train_data, Y_test_data = train_test_split(Xa, Y, test_size=0.2, random_state=3)

In [21]:
a = pd.concat([Xa,Y],axis = 1)
a['Target'] = Y.to_list()

In [40]:
Y_train_data.value_counts(normalize=True)

0    0.973699
1    0.026301
Name: T2, dtype: float64

In [19]:
Y_test_data.value_counts(normalize=True)

0    0.970453
1    0.029547
Name: T2, dtype: float64

# start

In [41]:
X_train = X_train_data.to_numpy()
Y_train = Y_train_data.to_numpy().flatten()
X_test = X_test_data.to_numpy()
Y_test = Y_test_data.to_numpy().flatten()

In [42]:
p= 0.6
entropy([p, 1-p],base = 2)

0.9709505944546688

In [43]:
def LR_cv3():
    grid={"C":[10**i for i in range(-3,3)], "penalty":["l1","l2"],"solver":["liblinear"],"random_state":[0]}
    logreg = LogisticRegression(C = 1)
    logreg_cv = GridSearchCV(estimator = logreg, param_grid = grid, scoring="roc_auc", cv = 3)
    return logreg_cv

def LR():
    logreg = LogisticRegression(C = 1)
    return logreg

def XGB():
    xgb = xgboost.XGBClassifier()
    return xgb


class cluster_predict():
    def __init__(self, n_cluster, c_cluster, model):
        self.n_cluster = n_cluster
        self.c_cluster = c_cluster
        self.model= model
        
    def fit(self, X, Y):
        self.cluster = KMedoids(n_clusters = self.n_cluster)
        if self.c_cluster == "AC":
            self.cluster.fit(X[:60000])
        elif self.c_cluster == "PC":
            self.cluster.fit(X[np.where(Y == 1)[0]])
        else:
            raise ValueError("Wrong type of c_cluster method is defined.")
        #print(pd.Series(self.cluster.predict(X)).value_counts())
    def cross_tab(self, X, Y):
        #re_clust = pd.DataFrame([cluter.predict(X_train), Y_train], index = ["C", "Y"]).T
        cross_tab = pd.crosstab(self.cluster.predict(X), Y).to_numpy()
        prob_lst = cross_tab[:,1]/np.sum(cross_tab, axis = 1)
        return prob_lst
    
    def entropy(self, X, Y):
        #re_clust = pd.DataFrame([cluter.predict(X_train), Y_train], index = ["C", "Y"]).T
        cross_tab = pd.crosstab(self.cluster.predict(X), Y).to_numpy()
        en = 0
        for i in range(cross_tab.shape[0]):
            p = cross_tab[i,1]/np.sum(cross_tab[i,:])
            en += (np.sum(cross_tab[i,:])/len(X))*(entropy([p, 1-p],base = 2))
        return en
    
    def predict(self, X_train, Y_train, X_test, Y_test):
        lab_tr = self.cluster.predict(X_train)
        lab_ts = self.cluster.predict(X_test)

        #model = LogisticRegression()
        #model.fit(X_train, Y_train)
        #re_train = [roc_auc_score(Y_train, model.predict_proba(X_train)[:,1])]
        #re_test = [roc_auc_score(Y_test, model.predict_proba(X_test)[:,1])]
        re_train = np.zeros(len(Y_train))
        re_test = np.zeros(len(Y_test))
        for i in range(self.n_cluster):
            posi_tr = np.where(lab_tr == i)[0]
            posi_ts = np.where(lab_ts == i)[0]
            if self.model == "LR":
                model = LR()
            elif self.model == "XGB":
                model = XGB()
            else:
                raise ValueError("Wrong type of model is defined.")
                
            model.fit(X_train[posi_tr], Y_train[posi_tr])
            
            re_train[posi_tr] = model.predict_proba(X_train[posi_tr])[:,1]
            re_test[posi_ts] = model.predict_proba(X_test[posi_ts])[:,1]

        return re_train, re_test

# Rescalor

In [45]:
res_time_lst = []
res = []
tr_lst = []
ts_lst = []


for p in [1,2,3]:
    poly = PolynomialFeatures(p, include_bias=False)
    X_poly_train = np.append(poly.fit_transform(X_train[:,:10]), X_train[:,10:],axis=1)
    X_poly_test = np.append(poly.fit_transform(X_test[:,:10]), X_test[:,10:],axis=1)


    lm = LinearRegression()
    time_start = time.time()
    lm.fit(X_poly_train, Y_train)
    time_end = time.time()
    res_time_lst.append(time_end - time_start)
    w_lm = lm.coef_
    res.append(w_lm)
    X_lm_tr = X_poly_train * w_lm
    X_lm_ts = X_poly_test * w_lm


    #grid_result = LR_cv3().fit(X_train, Y_train)
    lr = LogisticRegression()#**grid_result.best_params_)
    time_start = time.time()
    lr.fit(X_poly_train, Y_train)
    time_end = time.time()
    res_time_lst.append(time_end - time_start)
    w_lr = lr.coef_[0]
    res.append(w_lr)
    X_lr_tr = X_poly_train * w_lr
    X_lr_ts = X_poly_test * w_lr

    time_start = time.time()
    w_ig =  mutual_info_classif(X_poly_train,Y_train, random_state = 0)
    time_end = time.time()
    res_time_lst.append(time_end - time_start)
    res.append(w_ig)
    X_ig_tr = X_poly_train * w_ig
    X_ig_ts = X_poly_test * w_ig


    tr = [X_poly_train, X_lm_tr, X_lr_tr, X_ig_tr]
    ts = [X_poly_test, X_lm_ts, X_lr_ts, X_ig_ts]
    tr_lst.append(tr)
    ts_lst.append(ts)

c:\Users\king0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\king0\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

In [46]:
with pd.ExcelWriter('rescalor.xlsx') as w:
    pd.DataFrame({'REG':res[0], 'LR':res[1], 'IG':res[2]}).T.to_excel(w, 'identity')
    pd.DataFrame({'REG':res[3], 'LR':res[4], 'IG':res[5]}).T.to_excel(w, 'quadratic')
    pd.DataFrame({'REG':res[6], 'LR':res[7], 'IG':res[8]}).T.to_excel(w, 'cubic')
    #w.save()

In [47]:
res_std = []
for i in range(len(res)):
    res_std.append((res[i]-np.mean(res[i]))/np.std(res[i]))

In [48]:
res_lst = []
for a in ['Identity','Quadratic','Cubic']:
    for b in ['REG', 'LR', 'IG']:
        res_lst.append(a)
        res_lst.append(b)

In [49]:
res_f = np.array([])
for i in res_std:
    res_f = np.append(res_f, i)

In [50]:
res_var = np.array([])
res_n = np.array([])
res_poly = np.array([])
for i in [12, 67, 287]:
    for _ in range(3):
        res_var = np.append(res_var, [j for j in range(i)])
    for n in ['REG', 'LR', 'IG']:
        res_n = np.append(res_n, [n for j in range(i)])
res_poly = np.append(res_poly, np.repeat('Identity',36))
res_poly = np.append(res_poly, np.repeat('Quadratic',201))
res_poly = np.append(res_poly, np.repeat('Cubic',861))

In [28]:
pd.DataFrame({'REG':res[0],'LR':res[1],'IG':res[2]}).corr()

,REG,LR,IG
REG,1.000000,0.796982,0.042651
LR,0.796982,1.000000,0.085904
IG,0.042651,0.085904,1.000000


In [29]:
pd.DataFrame({'REG':res[3],'LR':res[4],'IG':res[5]}).corr()

,REG,LR,IG
REG,1.000000,0.731562,-0.051923
LR,0.731562,1.000000,-0.221914
IG,-0.051923,-0.221914,1.000000


In [30]:
pd.DataFrame({'REG':res[6],'LR':res[7],'IG':res[8]}).corr()

,REG,LR,IG
REG,1.000000,0.352069,-0.024565
LR,0.352069,1.000000,-0.173651
IG,-0.024565,-0.173651,1.000000


# PD and Entropy

In [ ]:
clt_lst = []

k = [i for i in range(1,10)]

for t in ['Train', 'Test']:
    for a in ['Identity', 'Quadratic', 'Cubic']:
        for b in ['All', 'Positive']:
            for c in ['EW', 'REG', 'LR', 'IG']:
                for d in k:
                    clt_lst.append(t)
                    clt_lst.append(a)
                    clt_lst.append(b)
                    clt_lst.append(c)
                    clt_lst.append(d)
clt_comb = np.array(clt_lst).reshape(-1,5)

In [32]:
method = ['AC', 'PC']
clt_time = []
tr_en = []
ts_en = []
tr_pd = []
ts_pd = []

for x_tr, x_ts in zip(tr_lst, ts_lst):
    for m in method:
        for X_tr, X_ts in zip(x_tr, x_ts):
            for i in k:
                cp = cluster_predict(n_cluster = i, c_cluster = m, model = "LR")
                time_start = time.time()
                cp.fit(X_tr, Y_train)
                time_end = time.time()
                if k == 1:
                    clt_time.append(0)
                else:
                    clt_time.append(time_end - time_start)
                

                tr_en.append(cp.entropy(X_tr, Y_train))
                ts_en.append(cp.entropy(X_ts, Y_test))
                tr_pd.append(cp.cross_tab(X_tr, Y_train))
                ts_pd.append(cp.cross_tab(X_ts, Y_test))
                

# Log-likelihood and AUC

In [51]:
ctp_lst = []

k = [i for i in range(4,5)] #change

for t in ['Train', 'Test']:

    for a in ['Identity', 'Quadratic', 'Cubic']:
        for b in ['All', 'Positive']:
            for c in ['EW', 'REG', 'LR', 'IG']:
                for d in k:
                    for e in ['LR', 'XGB']:
                        ctp_lst.append(t)
                        ctp_lst.append(a)
                        ctp_lst.append(b)
                        ctp_lst.append(c)
                        ctp_lst.append(d)
                        ctp_lst.append(e)

ctp_comb = np.array(ctp_lst).reshape(-1,6)

In [52]:
method = ['AC', 'PC']
model = ['LR', 'XGB']
res_dic = {0:'EW', 1:'REG',2:'LR',3:'IG'}
ctp_time = []
tr_ll = []
ts_ll = []
tr_auc = []
ts_auc = []
tr_bs = []
ts_bs = []

for x_tr, x_ts in zip(tr_lst, ts_lst):
    for cm in method:
        for s, (X_tr, X_ts) in enumerate(zip(x_tr, x_ts)):
            for i in k:
                for m in model:
                    cp = cluster_predict(n_cluster =i, c_cluster = cm, model = m)
                    cp.fit(X_tr, Y_train)

                    time_start = time.time()
                    y_pred_tr, y_pred_ts = cp.predict(X_tr,Y_train,X_ts,Y_test)
                    time_end = time.time()
                    
                    pd.DataFrame({'pred':y_pred_tr}).to_csv('Train_'+cm+'_'+res_dic[s]+'_'+str(i)+'_'+m+'.csv')
                    pd.DataFrame({'pred':y_pred_ts}).to_csv('Test_'+cm+'_'+res_dic[s]+'_'+str(i)+'_'+m+'.csv')
                    
                    ctp_time.append(time_end - time_start)
                    
                    tr_bs.append(brier_score_loss(Y_train, y_pred_tr))
                    ts_bs.append(brier_score_loss(Y_test, y_pred_ts))

                    tr_auc.append(roc_auc_score(Y_train, y_pred_tr))
                    ts_auc.append(roc_auc_score(Y_test, y_pred_ts))

                    tr_ll.append(log_loss(Y_train, y_pred_tr))
                    ts_ll.append(log_loss(Y_test, y_pred_ts))

In [ ]:
clt_col = pd.DataFrame(clt_comb, columns = ['Data', 'Feature', 'Cluster','Rescalor', 'K'])
ctp_col = pd.DataFrame(ctp_comb, columns = ['Data', 'Feature', 'Cluster','Rescalor', 'K', 'Model'])
clt_col = clt_col[clt_col['Data']=='Train']
ctp_col = ctp_col[ctp_col['Data']=='Train']

In [37]:
# with pd.ExcelWriter('time_output.xlsx') as w:
#     pd.concat([pd.DataFrame(np.array(res_lst).reshape(-1,2), columns = ['Feature', 'Rescalor']),pd.DataFrame({'Time':res_time_lst})],axis=1).to_excel(w, 'rescalor')
#     pd.concat([clt_col, pd.DataFrame({'Time':clt_time})], axis = 1).to_excel(w, 'cluster')
#     pd.concat([ctp_col, pd.DataFrame({'Time':ctp_time})], axis = 1).to_excel(w, 'classification')

In [38]:
# import csv
# with open('positive_rate_tr.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)
#     # write the data
#     writer.writerow(['Data', 'Feature', 'Cluster','Rescalor', 'K'] + ['PD'+str(i) for i in range(9)])
#     for i in range(len(tr_pd)):
#         writer.writerow(list(np.append(clt_col.iloc[i,:].to_numpy(), tr_pd[i])))
#         #writer.writerows(i)

# with open('positive_rate_ts.csv', 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)
#     # write the data
#     writer.writerow(['Data', 'Feature', 'Cluster','Rescalor', 'K'] + ['PD'+str(i) for i in range(9)])
#     for i in range(len(ts_pd)):
#         writer.writerow(list(np.append(clt_col.iloc[i,:].to_numpy(), ts_pd[i])))

In [ ]:
with pd.ExcelWriter('output(k=4).xlsx') as w:
#     pd.concat([pd.DataFrame(clt_comb, columns = ['Data', 'Feature', 'Cluster','Rescalor', 'K']), pd.DataFrame({'Entropy':np.append(tr_en, ts_en)})],axis = 1).to_excel(w, 'entropy')
    pd.concat([pd.DataFrame(ctp_comb, columns = ['Data', 'Feature', 'Cluster','Rescalor', 'K', 'Model']), pd.DataFrame({'LL':np.append(tr_ll, ts_ll), 'AUC':np.append(tr_auc, ts_auc), 'BS':np.append(tr_bs, ts_bs)})],axis = 1).to_excel(w, 'auc')